In [15]:
import numpy as np
import cv2



In [20]:
class BackGroundSubtractor:
# When constructing background subtractor, we
# take in two arguments:
# 1) alpha: The background learning factor, its value should
# be between 0 and 1. The higher the value, the more quickly
# your program learns the changes in the background.
# 2) firstFrame: This is the first frame from the video/webcam.
    def __init__(self,alpha,firstFrame):
        self.alpha  = alpha
        self.backGroundModel = firstFrame

    def getForeground(self,frame):
        # apply the background averaging formula:
        self.backGroundModel =  frame * self.alpha + self.backGroundModel * (1 - self.alpha)

# after the previous operation, the dtype of
# self.backGroundModel will be changed to a float type
# therefore we do not pass it to cv2.absdiff directly,
# instead we acquire a copy of it in the uint8 dtype
# and pass that to absdiff.

        return cv2.absdiff(self.backGroundModel.astype(np.uint8),frame)

cam = cv2.VideoCapture('data.mp4')

# Just a simple function to perform
# some filtering before any further processing.
def denoise(frame):
    frame = cv2.medianBlur(frame,5)
    frame = cv2.GaussianBlur(frame,(5,5),0)
    
    return frame

ret,frame = cam.read()
if ret is True:
    backSubtractor = BackGroundSubtractor(0.15,denoise(frame))
    run = True
else:
    run = False

while(run):
    # Read a frame from the camera
    ret,frame = cam.read()

    # If the frame was properly read.
    if ret is True:
        # Show the filtered image
        cv2.imshow('input',frame)

        # get the foreground
        foreGround = backSubtractor.getForeground(denoise(frame))

        # Apply thresholding on the background and display the resulting mask
        ret, mask = cv2.threshold(foreGround, 15, 255, cv2.THRESH_BINARY)

        cv2.imshow('mask',mask)

        key = cv2.waitKey(30) & 0xFF
    else:
        break

    if key == 27:
        break

cam.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 